In [ ]:
#Authors: Tlachac, et al
#Paper: "Automated Construction of Lexicons to Improve Depression Screening with Text Messages"

from empath import Empath
from empath import helpers as util
import pandas as pd
import numpy as np
import re
import string

In [ ]:
#create list of all empath categories

lexicon = Empath()
emp = lexicon.analyze("Testing", normalize=True)
wordlist = []
for word, value in emp.items():
    wordlist.append(word)
print(wordlist)

In [ ]:
# read the data frame 
dataFrame = pd.read_csv("sent14days.csv")
#print(dataFrame)

# obtain ID, texts, scores, and address 2 from dataframe
dataFrame = dataFrame[['id', 'body2', 'scores', 'address2']]
dataFrame.head()

In [ ]:
# Group data frame by participant ID 
def sortList(inList):
    newList = " ".join(inList)
    return newList

def scoreList(inList): 
    #print(list(inList))
    if len(inList) == 0: 
        return None
    else: 
        return list(inList)[0]

#function to clean out punctuation and make text lowercase
def cleanText(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation,"")
    return text.lower()

newCol = dataFrame[['id', 'body2']].groupby(by = "id").agg(len)

dataFrame = dataFrame.sort_values(by="id")
dataFrame = dataFrame.groupby(by = "id").agg({"body2":sortList, "scores": scoreList})
dataFrame['NumTexts'] = newCol

#only 2+ messages
dataFrame = dataFrame[dataFrame['NumTexts'] >= 2]

dataFrame.head()

In [ ]:
dataFrame.reset_index(inplace=True)

#actively remove punctuation and make lower case for text messages
dataFrame["body2"] = dataFrame["body2"].apply(cleanText)

dataFrame.head()

In [ ]:
for j in range(0, len(wordlist)):

    frequencies = []     
        
    #walk through each participant
    for i in range(0,dataFrame.shape[0]):
        body = dataFrame.body2[i]
        #n words in body
        bodylen = len(body.split(" "))
    
        emp = lexicon.analyze(body, categories=[wordlist[j]], normalize = True)

        for word, value in emp.items():
            print(word)
            print(value)
            frequencies.append(value)
    dataFrame[wordlist[j]] = frequencies

In [ ]:
dataFrame.head()

In [ ]:
dataFrame.to_csv("featuresDefault.csv")